In [2]:
import os, sys
from utils.sunrgbd_config import SUNRGBD_CONFIG
from configs.data_config import NYU40CLASSES
from utils.sunrgbd_utils import readsunrgbdframe, process_sunrgbd_frame, check_bdb
from utils.vis_tools_sunrgbd import Scene3D_SUNRGBD
import os
import numpy as np
import pickle
from configs.data_config import Config
from utils.sunrgbd_utils import proj_from_point_to_2d, get_corners_of_bb3d_no_index
from net_utils.libs import get_iou
from libs.tools import camera_cls_reg_sunrgbd, layout_size_avg_residual, ori_cls_reg, obj_size_avg_residual, bin_cls_reg, list_of_dict_to_dict_of_list
import json

sunrgbd_config = SUNRGBD_CONFIG()
sequence = readsunrgbdframe(sunrgbd_config, image_id=2)


scene = Scene3D_SUNRGBD(sequence)
# scene.draw_image()
# scene.draw_cls()
# scene.draw_inst()
im = scene.draw_projected_bdb3d(return_plt=True)

%matplotlib widget

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(im)

im, bdb2D_from_3D = scene.draw_projected_layoutbdb3d(return_plt=True, fix_polygon=True)
plt.subplot(122)
plt.imshow(im)
# scene.draw3D()

# display(fig)
plt.show()

[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_layoutbdb3d] Returned.


In [52]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from utils.sunrgbd_config import SUNRGBD_CONFIG
sunrgbd_config = SUNRGBD_CONFIG()

import pickle
with open(sunrgbd_config.obj_avg_size_file, 'rb') as f:
    obj_avg_size = pickle.load(f, encoding='latin1')


from utils.sunrgbd_utils import readsunrgbdframe, process_sunrgbd_frame

data_frame = readsunrgbdframe(sunrgbd_config, image_id=2)
scene = Scene3D_SUNRGBD(data_frame)

In [66]:
data_frame._manhattan_layout
def vis_cube_plt(Xs, ax, color='r'):
    index1 = [0, 1, 2, 3, 0, 4, 5, 6, 7, 4]
    index2 = [[1, 5], [2, 6], [3, 7]]
    ax.scatter3D(Xs[:, 0], Xs[:, 1], Xs[:, 2])
    ax.plot3D(Xs[index1, 0], Xs[index1, 1], Xs[index1, 2], color=color)
    for index in index2:
        ax.plot3D(Xs[index, 0], Xs[index, 1], Xs[index, 2], color=color)
    
# https://stackoverflow.com/questions/11140163/plotting-a-3d-cube-a-sphere-and-a-vector-in-matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("auto")

# vis_cube_plt(data_frame.manhattan_layout, ax)
bdb3d_layout = get_corners_of_bb3d_no_index(scene.layout['basis'], scene.layout['coeffs'], scene.layout['centroid'])
vis_cube_plt(bdb3d_layout, ax, 'k')

from utils.sunrgbd_utils import proj_from_point_to_2d
for bdb3d in scene.bdb3d:
    bdb3d_corners = get_corners_of_bb3d_no_index(bdb3d['basis'], bdb3d['coeffs'], bdb3d['centroid'])
    vis_cube_plt(bdb3d_corners, ax)

# draw cube
# for s, e in combinations(data_frame._manhattan_layout, 2):
#     ax.plot3D(*zip(s, e), color="b")

# plt.show()
fig.canvas.toolbar_visible = True

# display(fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
scene.layout

{'centroid': array([ 1.4042413 ,  1.34581369, -0.38624769]),
 'coeffs': array([2.30113326, 2.65418631, 2.21155161]),
 'basis': array([[ 0.98055391,  0.        ,  0.19624992],
        [ 0.        ,  1.        ,  0.        ],
        [-0.19624992,  0.        ,  0.98055391]])}

In [55]:
center_from_3D

array([[-1.26924317e+04, -1.33977421e+07]])

In [46]:
data_frame.bdb3d[1]

(array([[ 0.96835519, -0.24957608,  0.        ],
       [ 0.24783325,  0.9688027 ,  0.        ],
       [ 0.        ,  0.        ,  1.        ]]), array([[0.45907792, 0.34032744, 0.393974  ]]), array([[ 1.30276515,  3.05095168, -0.86269307]]), array(['night_stand'], dtype='<U11'), array(['SUNRGBD/kv2/kinect2data/000003_2014-05-26_14-24-42_260595134347_rgbf000040-resize'],
      dtype='<U81'), array([[-0.24783325, -0.9688027 ,  0.        ]]), array([[1]], dtype=uint8))

In [30]:
# # draw sphere
# u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# x = np.cos(u)*np.sin(v)
# y = np.sin(u)*np.sin(v)
# z = np.cos(v)
# ax.plot_wireframe(x, y, z, color="r")

# # draw a point
# ax.scatter([0], [0], [0], color="g", s=100)

# # draw a vector
# from matplotlib.patches import FancyArrowPatch
# from mpl_toolkits.mplot3d import proj3d


# class Arrow3D(FancyArrowPatch):

#     def __init__(self, xs, ys, zs, *args, **kwargs):
#         FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
#         self._verts3d = xs, ys, zs

#     def draw(self, renderer):
#         xs3d, ys3d, zs3d = self._verts3d
#         xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
#         self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
#         FancyArrowPatch.draw(self, renderer)

# a = Arrow3D([0, 1], [0, 1], [0, 1], mutation_scale=20,
#             lw=1, arrowstyle="-|>", color="k")
# ax.add_artist(a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …